In [4]:
import pyodbc
import pandas as pd
import sqlalchemy 

# Your existing connection function
def connect_to_sql_server(server, database, user, password):
    """Connects to a SQL Server database and returns a connection object."""
    conn_str = (
        r'DRIVER={SQL Server};'
        r'SERVER=' + server + ';'
        r'DATABASE=' + database + ';'
        r'UID=' + user + ';'
        r'PWD=' + password + ';'
    )
    conn = pyodbc.connect(conn_str)
    return conn

def execute_query(engine, query):
    """Executes a SQL query using SQLAlchemy and returns the results as a list of tuples."""
    with engine.connect() as conn:  # Establish a connection using the engine
        result = conn.execute(query)  # Execute the query

        # Get column names
        columns = result.keys()  # Get column names from the result object
        print("Columns returned by the query:", columns)

        # Fetch all rows
        rows = result.fetchall()

        # Print the shape of the results for debugging
        print("Shape of results:", len(rows), "rows,", len(rows[0]) if rows else 0, "columns")

        # Print the first few rows of the results
        print("First few rows of results:", rows[:5])  # Adjust the number of rows as needed

    return rows  # Return the fetched rows

# Establish connection using SQLAlchemy
server = '10.10.11.241'
database = 'omar.rme1'
user = 'omar'
password = 'omar123'
connection_string = f'mssql+pyodbc://{user}:{password}@{server}/{database}?driver=SQL+Server'
engine = create_engine(connection_string)

try:
    # Check if the connection is successful
    with engine.connect() as conn:  # Use a context manager to handle the connection
        print("Connected to SQL Server successfully!")

except Exception as e:
    print("Error connecting to SQL Server:", e)

Connected to SQL Server successfully!


In [5]:
# SQL query to sum amounts for each project, handling mixed data types
query = """
SELECT project_no, SUM(amount) AS TotalAmount
FROM [omar.rme1].[dbo].[cost_dist]
WHERE project_no IN ('144', '173', '172', '184', '198')  -- Enclose project numbers in single quotes
GROUP BY project_no;
"""

try:
    # Execute the query using the engine, creating a new connection
    with engine.connect() as conn:
        results = execute_query(conn, query)

    # Create a DataFrame from the results, using the column names from the query output
    df = pd.DataFrame(results, columns=['project_no', 'TotalAmount'])

    # Display the DataFrame
    print(df)

except Exception as e:
    print("Error executing query:", e)

Columns returned by the query: RMKeyView(['project_no', 'TotalAmount'])
Shape of results: 5 rows, 2 columns
First few rows of results: [('144', 856974103.9199996), ('172', 596670367.19), ('173', 42772299.35000001), ('184', 300843751.0199999), ('198', 122307643.32)]
  project_no   TotalAmount
0        144  8.569741e+08
1        172  5.966704e+08
2        173  4.277230e+07
3        184  3.008438e+08
4        198  1.223076e+08


In [6]:
# 1. SQL query to fetch project_no and project_name
query_names = """
SELECT DISTINCT project_no, project_name 
FROM [omar.rme1].[dbo].[cost_dist]
WHERE project_no IN ('144', '173', '172', '184', '198'); 
"""

# 2. Execute the query using pd.read_sql and the connection string
df_names = pd.read_sql(query_names, connection_string) 

# 3. Merge DataFrames using 'project_no' as the key
df_merged = pd.merge(df, df_names, on='project_no', how='left')

# 4. Fill in missing values with 0
df_merged.fillna(0, inplace=True)

# 5. Combine project_no and project_name for labels
df_merged['Project'] = df_merged['project_no'] + ' - ' + df_merged['project_name']

# 6. Calculate total cost
total_cost = df_merged['TotalAmount'].sum()

# 7. Create bar chart with adjusted width and project numbers in labels
chart = alt.Chart(df_merged).mark_bar().encode(
    x=alt.X('Project:N', axis=alt.Axis(title='Project')),
    y=alt.Y('TotalAmount:Q', axis=alt.Axis(title='Total Amount')),
    tooltip = ['Project', 'TotalAmount']
).properties(
    title=f'Total Amount by Project (Total: {total_cost:.2f})',
    width=800
).interactive()

# 8. Display chart
chart.save('total_amount_by_project_bar_chart.json')
chart.display()

ImportError: Using URI string without sqlalchemy installed.